# Zusammenfassung von Userplots

## Imports aus pysrl

In [ ]:
import os
import config
import warnings
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf

from pysrl.preparer.prep_fcts import load_learn_pages, load_transformed_data
from pysrl.preparer.prep_help_fcts import get_user_dfs, plot_lines
from pysrl.preparer.prep_help_fcts import time_dependent_features as timefig
from pysrl.preparer.prep_fcts import time_dependent_sec_spent as spentfig
from pysrl.preparer.prep_plot_fcts import learn_types_lineplots
from pysrl.cluster.cluster_plot_fcts import significant_corrs_heatmap, \
    string_ratio_grid, get_success_moderators, summarize_feature_influences
from pysrl.classifier.classify_fcts import merge_pers_feature
from pysrl.config.constants import ROOT

plt.rcParams.update({'figure.max_open_warning': 0})  # we want many figures!
results_path = os.path.join(ROOT, 'plotting', 'results')
warnings.filterwarnings('default')  # below they are ignored, reset this here

## Laden der notwendigen Daten

In [ ]:
dfs = get_user_dfs(load_transformed_data())  # The dfs we need for all plots
sorted_pages = load_learn_pages('learn_pages_sorted')  # for one type of plot

## Jetzt die eigentlichen Plots

### Definiere Parameter

In [ ]:
kwargs = {'plot': True, 'save': True, 'n_min': 0}  # only save relevant here
save = kwargs['save']
n = 47 # Number of users to show

### Zeitlicher Verlauf der Lerntypen

In [ ]:
labels = ["Tests", "Übungen", "Lesen"]
mapper = [0, 1, 1, 2, 2, 2]
spath = os.path.join(results_path, 'LerntypenVerlauf')
figs1 = timefig(dfs[:n], labels, f_typ="LearnType", mapping=mapper, 
                path=spath, **kwargs)

### Zeitlicher Verlauf der Schwierigkeit

In [ ]:
labels = ["Leicht", "Mittel", "Schwer"]
spath = os.path.join(results_path, 'SchwierigkeitVerlauf')
figs2= timefig(dfs[:n], labels, f_typ="difficulty", path=spath, **kwargs)

### Zeitlicher Verlauf der Korrektheit von Aufgaben

In [ ]:
labels = ["falsch", "teilw.", "richtig"]
spath = os.path.join(results_path, 'KorrektheitVerlauf')
figs3 = timefig(dfs[:n], labels, f_typ="ResponsTask", path=spath, **kwargs)

### Zeitlicher Verlauf der Testpunkte

In [ ]:
labels = ["Testpunkte"]
spath = os.path.join(results_path, 'TestpunkteVerlauf')
figs4 = timefig(dfs[:n], labels, f_typ="TestResQuant", path=spath, **kwargs)

### Zeitlicher Verlauf der Sekunden pro Seite

In [ ]:
labels = ["SekProSeite"]
figs5 = spentfig(dfs[:n], clabel=labels[0], save=save, plot=True,
                 cpath=results_path)

### Reihenfolge der Seiten im Vergleich zur Lernumgebung

In [ ]:
figs6 = []
for i, user_df in enumerate(dfs[:n]):
    spath = os.path.join(results_path, 'ActionOrder', str(i))
    figs6.append(plot_lines(user_df, sorted_pages, save=save, path=spath))

### Standard Lineplots der User

In [ ]:
learntypes = [[0], [2], [3], [5]]
spath = os.path.join(results_path, 'Lineplots')
figs7 = learn_types_lineplots(save=save, silent=True, file_extension='simple',
                              path=spath, dpi=300, learntypes=learntypes,
                              max_user=n)

In [ ]:
# With this all lineplots might be saved in one pdf, with lernerfolg annotated:

features = merge_pers_feature(rm_incomplete_user=False, fill_na=True)
annots = features[['User', 'NSt_Diff', 'RF_Diff']]

for i in range(len(figs7)):
    user = int(figs7[i][0]._suptitle.get_text())
    if user in annots['User'].values:
        nst = annots.loc[annots.User == user, 'NSt_Diff'].values[0]
        rf = annots.loc[annots.User == user, 'RF_Diff'].values[0]
        text = f'Lernerfolg -> NSt_Diff: {nst:.2f} | RF_Diff {rf:.2f}'
    else:
        text = f'Lernerfolg unbekannt, Tests unvollständig'
    figs7[i][1].annotate(text, xy=(100, 175), xycoords='figure pixels')
    
filename = os.path.join(results_path, 'lineplots_mit_lernerfolg.pdf')
pdf = matplotlib.backends.backend_pdf.PdfPages(filename)
for fig in [entry[0] for entry in figs7]:
    pdf.savefig(fig)
    
pdf.close()

### Significant Correlation Heatmaps

In [ ]:
kwargs = dict(method='pearson', p_threshold=5, min_cor=0, cor_threshold=0.8)

warnings.filterwarnings("ignore")
spath = os.path.join(results_path, 'Heatmaps')

fig8, _ = significant_corrs_heatmap(save=save, path=spath, **kwargs)

### String Similarity Clustermap

In [ ]:
grid = string_ratio_grid(save=save)

### Summary of Feature influences on Learn Success

In [ ]:
moderators1 = get_success_moderators(diff='RF_Diff', save=True)
fig9, _ = summarize_feature_influences(moderators1)

moderators2 = get_success_moderators(diff='NSt_Diff', save=True)
fig9, _ = summarize_feature_influences(moderators2)